# In this notebook we generate raw Ising configurations using Monte Carlo Metropolis Algorithm.

In [1]:
import numpy as np  # package for arrays
import matplotlib.pyplot as plt  # package for plotting
import time  # for timing
import random as ran
from time import time
# display plots inside of notebook
%matplotlib inline

Tc = 2.269

In [2]:
from numba.experimental import jitclass
from numba import jit

In [3]:
def init_random(shape, x):
    x = 1-x
    N_sites = np.prod(shape)
    n_missing = round(N_sites*x)
    n_occupied = N_sites - n_missing
    
    spins = [0 for _ in range(n_missing)] + [ran.choice([-1,1]) for _ in range(n_occupied)]
    flattened_state = np.array(ran.sample(spins, len(spins)))
    lattice_state = np.reshape(flattened_state, shape)
    
    return lattice_state

def init_up(shape, x): # randomise lattice with 75% up spins
    x = 1-x
    N_sites = np.prod(shape)
    n_missing = round(N_sites*x)
    n_occupied = N_sites - n_missing
    
    n_up = round(n_occupied*0.75)
    n_down = n_occupied - n_up
    spins = [0 for _ in range(n_missing)] + [1 for _ in range(n_up)] + [-1 for _ in range(n_down)]
    flattened_state = np.array(ran.sample(spins, len(spins)))
    lattice_state = np.reshape(flattened_state, shape)
    
    return lattice_state

def init_down(shape, x): # randomise lattice with 75% up spins
    x = 1-x
    N_sites = np.prod(shape)
    n_missing = round(N_sites*x)
    n_occupied = N_sites - n_missing
    
    n_down = round(n_occupied*0.75)
    n_up = n_occupied - n_down
    spins = [0 for _ in range(n_missing)] + [1 for _ in range(n_up)] + [-1 for _ in range(n_down)]
    flattened_state = np.array(ran.sample(spins, len(spins)))
    lattice_state = np.reshape(flattened_state, shape)
    
    return lattice_state


In [4]:
init_random((15,15), x=0.99)

array([[ 1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1],
       [-1,  1, -1, -1,  1,  1, -1,  1,  1, -1,  1,  1, -1,  1,  1],
       [-1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1],
       [ 1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1],
       [-1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1],
       [ 1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1],
       [-1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1,  1, -1, -1, -1],
       [-1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1],
       [-1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1, -1],
       [-1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1],
       [ 1,  1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1],
       [ 1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1,  1,  1,  0, -1],
       [-1,  1,  1, -1,  0, -1, -1,  1, -1, -1, -1, -1,  1, -1,  1],
       [ 1, -1,  1, -1,  1, -1, -1

In [4]:
# @jit
def flip_spin(lattice_state, newSpin, index):
    lattice_state[index] = newSpin
    return lattice_state

# @jit
def calculate_energy_of_spin(lattice_state, index, J=1):
    '''
     This works for dilute ising model too as if the nearest neighbor is empty, 
     then the spin is zero so it contributes nothing
    '''
    
    i,j = index
    spin_here = lattice_state[index]  # value of spin here
    shape = lattice_state.shape
    
    Il = (i, j-1)
    Ia = (i-1, j)
    
    if i==shape[0]-1:
        Ib = (0,j)
    else:
        Ib = (i+1, j)
        
    if j==shape[1]-1:
        Ir = (i, 0)
    else:
        Ir = (i, j+1)
        
    Sa = lattice_state[Ia]
    Sb = lattice_state[Ib]
    Sr = lattice_state[Ir]
    Sl = lattice_state[Il]
    
    E = -J*spin_here*(Sa + Sb + Sr + Sl)
    return E

# @jit
def occupied_indices(lattice_state):
    occupied_sites = []
    for i,row in enumerate(lattice_state):
        for j,spin in enumerate(row):
            if spin:
                occupied_sites.append((i,j))
    return occupied_sites

# @jit
def trial(lattice_state, occupied_sites, kT):

    # generate random index
    index = ran.choice(occupied_sites)

    # old spin and proposed new spin
    s_old = lattice_state[index]
    s_trial = s_old*-1

    # Calculate current energy of spin
    E_old = calculate_energy_of_spin(lattice_state, index)

    # Propose flip
    lattice_state = flip_spin(lattice_state, s_trial, index)

    # Calculate new energy of spin
    E_new = calculate_energy_of_spin(lattice_state, index)

    # Probability of accepting
    P =  np.exp(-(E_new - E_old) / kT)

    if ran.random() > P : #reject
        return flip_spin(lattice_state, s_old, index) # flip spin back

    return lattice_state

# @jit
def run_N_trials(lattice_state, occupied_sites, N, kT):
    for e in range(N):
        lattice_state = trial(lattice_state, occupied_sites, kT)
    return lattice_state
        
# @jit 
def run_N_sweeps(lattice_state, N_sweeps, kT):
    occupied_sites = occupied_indices(lattice_state)
    sweep_size = np.count_nonzero(lattice_state)
    for _ in range(N_sweeps):
        for _ in range(sweep_size):
            lattice_state = trial(lattice_state, occupied_sites, kT)
    return lattice_state    

# @jit
def magnetisation(state):
    N_spins = np.count_nonzero(state)
    return np.sum(state)/N_spins

# @jit
def test_convergence(lattice_state, N_trials, kT):
    occupied_sites = occupied_indices(lattice_state)
    m = []
    for _ in range(N_trials):
        lattice_state = trial(lattice_state, occupied_sites, kT)
        m.append(magnetisation(lattice_state))
    return m

### Functions to analyse data

In [5]:
def magnetisation(state):
    N_spins = np.count_nonzero(state)
    return np.sum(state)/N_spins

def flatten_states(states):
    '''
    states must have the shape: (n_states, dimension1, dimension2, ... )
    '''
    shape = states.shape
    n_states = shape[0]
    N = np.prod(np.array(list(shape)[1:]))
    newshape = (n_states, N)
    
    return np.reshape(states, newshape)

def m_from_states(states):
    m = []
    for state in states:
        m.append(magnetisation(state))
    return np.array(m)

def mean_absolute(value, y, central_limit=True):
    temperatures = list(set(y))
    
    means = []
    errs = []
    for kT in temperatures:
        filt = y==kT
        samples = value[filt]
        N_samples = len(samples)
        
        samples = np.abs(samples)
        mean = np.mean(samples)
        if central_limit:
            err = np.std(samples)/np.sqrt(N_samples)
        else:
            err = np.std(samples)
        
        errs.append(err)
        means.append(mean)
        
    return means, errs, temperatures

def test_convergence(lattice_state, N_trials, kT):
    occupied_sites = occupied_indices(lattice_state)
    m = []
    for _ in range(N_trials):
        lattice_state = trial(lattice_state, occupied_sites, kT)
        m.append(magnetisation(lattice_state))
    return m

def show_state(lattice_state):
    img = lattice_state
    _, ax = plt.subplots()
    ax.imshow(img, cmap=plt.get_cmap('gray'), interpolation="nearest")

### Seeing what the states look like

In [7]:
J = 1
x = 1
shape = (64,64)
lattice = init_random(shape=shape, x=x)

In [8]:
n_sweeps = 1000

kT = 1

t1 = time()
lattice = run_N_sweeps(lattice, n_sweeps, kT=kT)
print(time() - t1)

19.662104845046997


### Testing convergence

In [284]:
# Initialize lattice
J = 1
x = 0
shape = (64,64)
lattice = init_random(shape=shape, x=x)

In [141]:
n_sweeps = 1000
sweep_size = np.count_nonzero(lattice)
n_trials = sweep_size*n_sweeps

t1 = time()

kT = 2.2
trial_count = 0
m = test_convergence(lattice, n_trials, kT=kT)
m = np.array(m)

print(time() - t1)

22.177244901657104


In [142]:
avM = []
for n in range(n_sweeps):
    avM.append( np.mean(m[0: sweep_size*(n+1)]) )

In [117]:
trial_count

0

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20,20))

axs[0].set_title('Magnetisation')

axs[0].scatter(range(n_trials), m)

axs[1].set_title('Average Magnetisation (cumulative)')
axs[1].scatter(range(n_sweeps), avM)

### Collecting Data

Data at the end

Must have into: X/y L x 

In [6]:
def collect_data(L, x, temp_dict, method, n_measurements=5, stop_limit=4000):
    if method=='up_down':
        path = 'data/up_down/'
    elif method=='random':
        path = 'data/random/'
        
    filepath = path+'L='+str(L)+'_x='+str(x)
    
    while True:
        
        try:
            X = np.loadtxt(filepath+'_X.txt')
            y = np.loadtxt(filepath+'_y.txt')
            X = list(X)
            y = list(y)
        
        except:
            X = []
            y = []
        
        states = []
        labels = []
        for _ in range(n_measurements):
            kT = ran.choice(list(temp_dict.keys()))
            burn_sweeps = temp_dict[kT]
            
            if method == 'random':
                lattice = init_random(shape=(L,L), x=x)
            if method == 'up_down':
                if ran.random() > 0.5:
                    lattice = init_up(shape=(L,L), x=x) # randomise lattice
                else:
                    lattice = init_down(shape=(L,L), x=x)
            lattice = run_N_sweeps(lattice, burn_sweeps, kT) # run sweeps
#             show_state(lattice)
#             print(lattice)
#             print(kT)
            states.append(lattice) # measure
            labels.append(kT)
        
        X = X + list(flatten_states(np.array(states)))
        y = y + labels
        X = np.array(X)
        y = np.array(y)
        
        print(X.shape)
        print(y.shape)
        if y.shape[0] > stop_limit:
            break
        
        print('saving...')
        np.savetxt(filepath+'_X.txt', X, fmt='%i')
        np.savetxt(filepath+'_y.txt', y, fmt='%1.5f')
        print('done saving')
        
def get_details(L, x):
    filepath = 'data/method_3/L='+str(L)+'_x='+str(x)+'_y.txt'
    y = np.loadtxt(filepath)
    temps = set(y)
    Tcount = {}
    for kT in temps:
        Tcount[kT] = np.count_nonzero(y==kT)
    return Tcount

In [15]:
{2.1: 100, 2.2: 100, 2.3: 100, 2.4: 100}

L = 50
x = 1
path = 'data/up_down/'
tDict = temp_dict
# filepath = path+'L='+str(L)+'_x='+str(x)
# with open(filepath+'_T.txt', 'w') as file:
#     file.write(str(tDict))

collect_data(L=L, x=x, temp_dict=tDict, method='up_down', n_measurements=10, stop_limit=2500)


(1850, 2500)
(1850,)
saving...
done saving
(1860, 2500)
(1860,)
saving...
done saving
(1870, 2500)
(1870,)
saving...
done saving
(1880, 2500)
(1880,)
saving...
done saving
(1890, 2500)
(1890,)
saving...
done saving
(1900, 2500)
(1900,)
saving...
done saving
(1910, 2500)
(1910,)
saving...
done saving
(1920, 2500)
(1920,)
saving...
done saving
(1930, 2500)
(1930,)
saving...
done saving
(1940, 2500)
(1940,)
saving...
done saving
(1950, 2500)
(1950,)
saving...
done saving
(1960, 2500)
(1960,)
saving...
done saving
(1970, 2500)
(1970,)
saving...
done saving


KeyboardInterrupt: 

In [7]:
def temp_dict_(Tc):
    Tc = round(Tc,1)
    low_Ts = np.array([-0.8,-0.7,-0.6,-0.5,-0.4]) + Tc
    critical_Ts = np.array([-0.3,-0.2,-0.1,0,0.1,0.2,0.3]) + Tc
    high_Ts = np.array([0.4,0.5,0.6,0.7,0.8]) + Tc
    
    temp_dict = {
        **{Tl: 50 for Tl in low_Ts},
        **{Tc: 100 for Tc in critical_Ts},
        **{Th: 70 for Th in high_Ts}
    }
    
    return temp_dict

In [10]:
def start(path, L, x, Tc):
    tDict = temp_dict_(Tc=Tc)
    filepath = path+'L='+str(L)+'_x='+str(x)
    with open(filepath+'_T.txt', 'w') as file:
        file.write(str(tDict))

    collect_data(L=L, x=x, temp_dict=tDict, method='up_down', n_measurements=10, stop_limit=1500)

In [14]:


start(path='data/up_down/', L=50, x=0.8, Tc=1.5)
start(path='data/up_down/', L=30, x=0.8, Tc=1.5)

start(path='data/up_down/', L=50, x=0.7, Tc=1)
start(path='data/up_down/', L=30, x=0.7, Tc=1)


(10, 2500)
(10,)
saving...
done saving
(20, 2500)
(20,)
saving...
done saving
(30, 2500)
(30,)
saving...
done saving
(40, 2500)
(40,)
saving...
done saving
(50, 2500)
(50,)
saving...
done saving
(60, 2500)
(60,)
saving...
done saving
(70, 2500)
(70,)
saving...
done saving
(80, 2500)
(80,)
saving...
done saving
(90, 2500)
(90,)
saving...
done saving
(100, 2500)
(100,)
saving...
done saving
(110, 2500)
(110,)
saving...
done saving
(120, 2500)
(120,)
saving...
done saving
(130, 2500)
(130,)
saving...
done saving
(140, 2500)
(140,)
saving...
done saving
(150, 2500)
(150,)
saving...
done saving
(160, 2500)
(160,)
saving...
done saving
(170, 2500)
(170,)
saving...
done saving
(180, 2500)
(180,)
saving...
done saving
(190, 2500)
(190,)
saving...
done saving
(200, 2500)
(200,)
saving...
done saving
(210, 2500)
(210,)
saving...
done saving
(220, 2500)
(220,)
saving...
done saving
(230, 2500)
(230,)
saving...
done saving
(240, 2500)
(240,)
saving...
done saving
(250, 2500)
(250,)
saving...
done 

(500, 900)
(500,)
saving...
done saving
(510, 900)
(510,)
saving...
done saving
(520, 900)
(520,)
saving...
done saving
(530, 900)
(530,)
saving...
done saving
(540, 900)
(540,)
saving...
done saving
(550, 900)
(550,)
saving...
done saving
(560, 900)
(560,)
saving...
done saving
(570, 900)
(570,)
saving...
done saving
(580, 900)
(580,)
saving...
done saving
(590, 900)
(590,)
saving...
done saving
(600, 900)
(600,)
saving...
done saving
(610, 900)
(610,)
saving...
done saving
(620, 900)
(620,)
saving...
done saving
(630, 900)
(630,)
saving...
done saving
(640, 900)
(640,)
saving...
done saving
(650, 900)
(650,)
saving...
done saving
(660, 900)
(660,)
saving...
done saving
(670, 900)
(670,)
saving...
done saving
(680, 900)
(680,)
saving...
done saving
(690, 900)
(690,)
saving...
done saving
(700, 900)
(700,)
saving...
done saving
(710, 900)
(710,)
saving...
done saving
(720, 900)
(720,)
saving...
done saving
(730, 900)
(730,)
saving...
done saving
(740, 900)
(740,)
saving...
done saving


(1000, 2500)
(1000,)
saving...
done saving
(1010, 2500)
(1010,)
saving...
done saving
(1020, 2500)
(1020,)
saving...
done saving
(1030, 2500)
(1030,)
saving...
done saving
(1040, 2500)
(1040,)
saving...
done saving
(1050, 2500)
(1050,)
saving...
done saving
(1060, 2500)
(1060,)
saving...
done saving
(1070, 2500)
(1070,)
saving...
done saving
(1080, 2500)
(1080,)
saving...
done saving
(1090, 2500)
(1090,)
saving...
done saving
(1100, 2500)
(1100,)
saving...
done saving
(1110, 2500)
(1110,)
saving...
done saving
(1120, 2500)
(1120,)
saving...
done saving
(1130, 2500)
(1130,)
saving...
done saving
(1140, 2500)
(1140,)
saving...
done saving
(1150, 2500)
(1150,)
saving...
done saving
(1160, 2500)
(1160,)
saving...
done saving
(1170, 2500)
(1170,)
saving...
done saving
(1180, 2500)
(1180,)
saving...
done saving
(1190, 2500)
(1190,)
saving...
done saving
(1200, 2500)
(1200,)
saving...
done saving
(1210, 2500)
(1210,)
saving...
done saving
(1220, 2500)
(1220,)
saving...
done saving
(1230, 2500

done saving
(1490, 900)
(1490,)
saving...
done saving
(1500, 900)
(1500,)
saving...
done saving
(1510, 900)
(1510,)
